In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from sklearn import multioutput
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dropout, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


df = pd.read_csv('final_df.csv')
df.tail(10)

,timestamp,total_market_cap,total_volume_24h,total_volume_24h_reported,altcoin_market_cap,altcoin_volume_24h,altcoin_volume_24h_reported,log_total_volume_24h,log_altcoin_volume_24h,log_val,...,value_classification,btc_dominance,active_cryptocurrencies,active_market_pairs,active_exchanges,log_market_pairs,log_val_diff,log_alt_diff,zscore_12,volume_ratio
8055,2025-04-18 23:20:00+00:00,2.660490e+12,4.427403e+10,2.679970e+11,9.857119e+11,3.189471e+10,2.007214e+11,24.513664,24.185706,24.513664,...,Fear,62.9500,9881,100644,812,11.519345,0.008873,0.008873,-0.974089,0.720393
8056,2025-04-18 23:25:00+00:00,2.660241e+12,4.464964e+10,2.682257e+11,9.853354e+11,3.192512e+10,2.009117e+11,24.522112,24.186659,24.522112,...,Fear,62.9607,9881,100644,812,11.519345,0.008448,0.008448,0.098086,0.715014
8057,2025-04-18 23:30:00+00:00,2.661569e+12,4.405892e+10,2.685039e+11,9.855550e+11,3.203574e+10,2.011864e+11,24.508794,24.190118,24.508794,...,Fear,62.9709,9881,100642,812,11.519325,-0.013318,-0.013318,-1.286152,0.727111
8058,2025-04-18 23:35:00+00:00,2.662044e+12,4.423893e+10,2.688271e+11,9.860132e+11,3.189938e+10,2.014782e+11,24.512871,24.185852,24.512871,...,Fear,62.9603,9881,100642,812,11.519325,0.004077,0.004077,-0.665538,0.721070
8059,2025-04-18 23:40:00+00:00,2.662647e+12,4.429146e+10,2.688528e+11,9.862613e+11,3.195669e+10,2.015366e+11,24.514058,24.187647,24.514058,...,Fear,62.9594,9881,100641,812,11.519315,0.001187,0.001187,-0.397445,0.721509
8060,2025-04-18 23:45:00+00:00,2.663589e+12,4.466339e+10,2.689120e+11,9.866787e+11,3.197045e+10,2.015831e+11,24.522420,24.188078,24.522420,...,Fear,62.9568,9881,100641,812,11.519315,0.008362,0.008362,0.852893,0.715809
8061,2025-04-18 23:50:00+00:00,2.663492e+12,4.458681e+10,2.690797e+11,9.870101e+11,3.189565e+10,2.017357e+11,24.520704,24.185736,24.520704,...,Fear,62.9430,9881,100641,812,11.519315,-0.001716,-0.001716,0.707825,0.715361
8062,2025-04-18 23:55:00+00:00,2.662779e+12,4.461812e+10,2.691684e+11,9.866987e+11,3.193180e+10,2.018412e+11,24.521406,24.186868,24.521406,...,Fear,62.9448,9881,100641,812,11.519315,0.000702,0.000702,0.907881,0.715669
8063,2025-04-19 00:00:00+00:00,2.663370e+12,4.467855e+10,2.693401e+11,9.867543e+11,3.195054e+10,2.020169e+11,24.522759,24.187455,24.522759,...,Fear,62.9509,9881,100641,812,11.519315,0.001354,0.001354,1.166323,0.715120
8064,2025-04-19 00:05:00+00:00,2.663696e+12,4.455600e+10,2.675956e+11,9.868494e+11,3.188297e+10,2.009301e+11,24.520013,24.185338,24.520013,...,Fear,62.9519,9881,100641,812,11.519315,-0.002747,-0.002747,0.611060,0.715571


In [48]:
cols_for_drop = ['log_val', 'log_alt_vol', 'log_alt_val', 'vol_mean_12', 'vol_std_12', 'alt_mean_12', 'alt_std_12', 'cap_mean_12', 'cap_std_12', 'value_classification']
df = df.drop(columns=cols_for_drop)
df.tail(10)

,timestamp,total_market_cap,total_volume_24h,total_volume_24h_reported,altcoin_market_cap,altcoin_volume_24h,altcoin_volume_24h_reported,log_total_volume_24h,log_altcoin_volume_24h,vol_pct_1,...,value,btc_dominance,active_cryptocurrencies,active_market_pairs,active_exchanges,log_market_pairs,log_val_diff,log_alt_diff,zscore_12,volume_ratio
8055,2025-04-18 23:20:00+00:00,2.660490e+12,4.427403e+10,2.679970e+11,9.857119e+11,3.189471e+10,2.007214e+11,24.513664,24.185706,0.000362,...,32.0,62.9500,9881,100644,812,11.519345,0.008873,0.008873,-0.974089,0.720393
8056,2025-04-18 23:25:00+00:00,2.660241e+12,4.464964e+10,2.682257e+11,9.853354e+11,3.192512e+10,2.009117e+11,24.522112,24.186659,0.000345,...,32.0,62.9607,9881,100644,812,11.519345,0.008448,0.008448,0.098086,0.715014
8057,2025-04-18 23:30:00+00:00,2.661569e+12,4.405892e+10,2.685039e+11,9.855550e+11,3.203574e+10,2.011864e+11,24.508794,24.190118,-0.000543,...,32.0,62.9709,9881,100642,812,11.519325,-0.013318,-0.013318,-1.286152,0.727111
8058,2025-04-18 23:35:00+00:00,2.662044e+12,4.423893e+10,2.688271e+11,9.860132e+11,3.189938e+10,2.014782e+11,24.512871,24.185852,0.000166,...,32.0,62.9603,9881,100642,812,11.519325,0.004077,0.004077,-0.665538,0.721070
8059,2025-04-18 23:40:00+00:00,2.662647e+12,4.429146e+10,2.688528e+11,9.862613e+11,3.195669e+10,2.015366e+11,24.514058,24.187647,0.000048,...,32.0,62.9594,9881,100641,812,11.519315,0.001187,0.001187,-0.397445,0.721509
8060,2025-04-18 23:45:00+00:00,2.663589e+12,4.466339e+10,2.689120e+11,9.866787e+11,3.197045e+10,2.015831e+11,24.522420,24.188078,0.000341,...,32.0,62.9568,9881,100641,812,11.519315,0.008362,0.008362,0.852893,0.715809
8061,2025-04-18 23:50:00+00:00,2.663492e+12,4.458681e+10,2.690797e+11,9.870101e+11,3.189565e+10,2.017357e+11,24.520704,24.185736,-0.000070,...,32.0,62.9430,9881,100641,812,11.519315,-0.001716,-0.001716,0.707825,0.715361
8062,2025-04-18 23:55:00+00:00,2.662779e+12,4.461812e+10,2.691684e+11,9.866987e+11,3.193180e+10,2.018412e+11,24.521406,24.186868,0.000029,...,32.0,62.9448,9881,100641,812,11.519315,0.000702,0.000702,0.907881,0.715669
8063,2025-04-19 00:00:00+00:00,2.663370e+12,4.467855e+10,2.693401e+11,9.867543e+11,3.195054e+10,2.020169e+11,24.522759,24.187455,0.000055,...,32.0,62.9509,9881,100641,812,11.519315,0.001354,0.001354,1.166323,0.715120
8064,2025-04-19 00:05:00+00:00,2.663696e+12,4.455600e+10,2.675956e+11,9.868494e+11,3.188297e+10,2.009301e+11,24.520013,24.185338,-0.000112,...,32.0,62.9519,9881,100641,812,11.519315,-0.002747,-0.002747,0.611060,0.715571


In [61]:
def create_all_features(df, rolling_windows=[12, 24, 288]):
    df = df.sort_values('timestamp').reset_index(drop=True)

    df['timestamp'] = pd.to_datetime(df['timestamp'])

    df['log_total_volume_24h_reported'] = np.log(df['total_volume_24h_reported'])
    df['log_altcoin_volume_24h_reported'] = np.log(df['altcoin_volume_24h_reported'])

    ts = df['timestamp']
    df['hour'] = ts.dt.hour
    df['dow'] = ts.dt.weekday
    df['month'] = ts.dt.month
    for col, period in [('hour', 24), ('dow', 7), ('month', 12)]:
        df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / period)
        df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / period)

    for w in rolling_windows:
        df[f'vol_mean_{w}'] = df['log_total_volume_24h'].rolling(w).mean()
        df[f'vol_std_{w}'] = df['log_total_volume_24h'].rolling(w).std()
        df[f'alt_mean_{w}'] = df['log_altcoin_volume_24h'].rolling(w).mean()
        df[f'alt_std_{w}'] = df['log_altcoin_volume_24h'].rolling(w).std()
        df[f'cap_mean_{w}'] = df['log_total_cap'].rolling(w).mean()
        df[f'cap_std_{w}'] = df['log_total_cap'].rolling(w).std()

    df = df.dropna().reset_index(drop=True)

    return df

In [62]:
new_df = create_all_features(df, rolling_windows=[12, 24, 288])
new_df['timestamp'] = pd.to_datetime(df['timestamp'])
new_df.tail(10)

,timestamp,total_market_cap,total_volume_24h,total_volume_24h_reported,altcoin_market_cap,altcoin_volume_24h,altcoin_volume_24h_reported,log_total_volume_24h,log_altcoin_volume_24h,vol_pct_1,...,alt_mean_24,alt_std_24,cap_mean_24,cap_std_24,vol_mean_288,vol_std_288,alt_mean_288,alt_std_288,cap_mean_288,cap_std_288
7768,2025-04-17 23:25:00+00:00,2.660490e+12,4.427403e+10,2.679970e+11,9.857119e+11,3.189471e+10,2.007214e+11,24.513664,24.185706,0.000362,...,24.192134,0.003948,28.611870,0.000898,24.754093,0.119875,24.358244,0.087844,28.611550,0.001126
7769,2025-04-17 23:30:00+00:00,2.660241e+12,4.464964e+10,2.682257e+11,9.853354e+11,3.192512e+10,2.009117e+11,24.522112,24.186659,0.000345,...,24.191863,0.004095,28.611770,0.001026,24.752850,0.120419,24.357320,0.088245,28.611536,0.001128
7770,2025-04-17 23:35:00+00:00,2.661569e+12,4.405892e+10,2.685039e+11,9.855550e+11,3.203574e+10,2.011864e+11,24.508794,24.190118,-0.000543,...,24.191712,0.004089,28.611694,0.001092,24.751603,0.121081,24.356411,0.088615,28.611522,0.001122
7771,2025-04-17 23:40:00+00:00,2.662044e+12,4.423893e+10,2.688271e+11,9.860132e+11,3.189938e+10,2.014782e+11,24.512871,24.185852,0.000166,...,24.191515,0.004257,28.611587,0.001117,24.750370,0.121698,24.355491,0.089005,28.611508,0.001114
7772,2025-04-17 23:45:00+00:00,2.662647e+12,4.429146e+10,2.688528e+11,9.862613e+11,3.195669e+10,2.015366e+11,24.514058,24.187647,0.000048,...,24.191420,0.004318,28.611497,0.001127,24.749103,0.122253,24.354576,0.089373,28.611497,0.001110
7773,2025-04-17 23:50:00+00:00,2.663589e+12,4.466339e+10,2.689120e+11,9.866787e+11,3.197045e+10,2.015831e+11,24.522420,24.188078,0.000341,...,24.191064,0.004222,28.611388,0.001069,24.747909,0.122782,24.353666,0.089729,28.611488,0.001106
7774,2025-04-17 23:55:00+00:00,2.663492e+12,4.458681e+10,2.690797e+11,9.870101e+11,3.189565e+10,2.017357e+11,24.520704,24.185736,-0.000070,...,24.190613,0.004187,28.611299,0.001035,24.746675,0.123275,24.352748,0.090091,28.611481,0.001105
7775,2025-04-18 00:00:00+00:00,2.662779e+12,4.461812e+10,2.691684e+11,9.866987e+11,3.193180e+10,2.018412e+11,24.521406,24.186868,0.000029,...,24.190135,0.003913,28.611195,0.000993,24.745451,0.123756,24.351842,0.090443,28.611472,0.001103
7776,2025-04-18 00:05:00+00:00,2.663370e+12,4.467855e+10,2.693401e+11,9.867543e+11,3.195054e+10,2.020169e+11,24.522759,24.187455,0.000055,...,24.189794,0.003766,28.611099,0.000928,24.744249,0.124233,24.350932,0.090775,28.611464,0.001101
7777,2025-04-18 00:10:00+00:00,2.663696e+12,4.455600e+10,2.675956e+11,9.868494e+11,3.188297e+10,2.009301e+11,24.520013,24.185338,-0.000112,...,24.189408,0.003726,28.611019,0.000868,24.743023,0.124698,24.350022,0.091117,28.611455,0.001097


In [68]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

new_df = new_df.sort_values('timestamp').reset_index(drop=True)

def learning_random_forest(horizon=1, df=new_df, random_state=42, n_iter_search=32):
    for h in range(1, horizon + 1):
        df[f'y{h}'] = df['log_total_volume_24h'].shift(-h) - df['log_total_volume_24h']
    target_cols = [f'y{h}' for h in range(1, horizon + 1)]

    drop_raw = [
        'total_market_cap', 'altcoin_market_cap',
        'total_volume_24h','altcoin_volume_24h',
        'total_volume_24h_reported', 'total_altcoin_volume_24h_reported',
        'active_market_pairs'
    ]

    feature_cols = [col for col in df.columns
                    if col not in ['timestamp'] + target_cols + drop_raw]

    X = df[feature_cols].to_numpy()
    y = df[target_cols].to_numpy()

    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.2,
        shuffle=False
    )

    scaler_x = MinMaxScaler().fit(X_train)
    X_train_s = scaler_x.transform(X_train)
    X_test_s = scaler_x.transform(X_test)

    base_model = RandomForestRegressor(random_state=random_state, n_jobs=-1)
    param_dist = {
        'n_estimators':    [100, 200, 500, 1000],
        'max_features':    ['sqrt','log2', 0.3, 0.5],
        'max_depth':       [None, 10, 20, 30],
        'min_samples_split':[2, 5, 10],
        'min_samples_leaf':[1, 2, 5, 10],
        'bootstrap':       [True, False]
    }
    tscv = TimeSeriesSplit(n_splits=5)
    search = RandomizedSearchCV(
        estimator=base_model,
        param_distributions=param_dist,
        n_iter=n_iter_search,
        cv=tscv,
        scoring='neg_mean_squared_error',
        random_state=random_state,
        verbose=1,
        n_jobs=-1
    )
    search.fit(X_train_s, y_train)
    print(f">> Best RF params: {search.best_params_}")
    print(f">> CV MAE: {-search.best_score_:.3f}")


    best_model = search.best_estimator_
    mor = MultiOutputRegressor(best_model)
    mor.fit(X_train_s, y_train)

    for i, h in enumerate(target_cols):
        imp = pd.Series(mor.estimators_[i].feature_importances_, index=feature_cols)
        print(f"===Horizion {h} ===")
        print(imp.sort_values(ascending=False).head(10))
        print()

    return {
        'model': mor,
        'scaler': scaler_x,
        'feature_cols': feature_cols,
        'best_params': search.best_params_,
        'X_test': X_test,
        'y_test': y_test,
    }


In [69]:
res = learning_random_forest(
    horizon=48,
    df=new_df,
    random_state=42,
    n_iter_search=50
)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
>> Best RF params: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'log2', 'max_depth': 10, 'bootstrap': False}
>> CV MAE: 0.006
===Horizion y1 ===
cap_std_12      0.074203
zscore_12       0.068295
vol_pct_1       0.059644
log_val_diff    0.059202
log_alt_diff    0.055514
alt_pct_1       0.055272
vol_std_12      0.048505
vol_mean_288    0.041124
alt_mean_288    0.039793
alt_std_12      0.038504
dtype: float64

===Horizion y2 ===
cap_std_12      0.077563
zscore_12       0.072592
vol_std_12      0.054861
alt_pct_1       0.050101
alt_mean_288    0.049867
vol_mean_288    0.048874
vol_pct_1       0.037534
alt_std_12      0.035976
vol_std_24      0.033798
log_alt_diff    0.032930
dtype: float64

===Horizion y3 ===
zscore_12       0.080133
cap_std_12      0.064662
vol_mean_288    0.052973
alt_pct_1       0.051865
alt_mean_288    0.049941
vol_std_12      0.048353
log_alt_diff    0.037671
alt_std

In [70]:
new_df.to_csv('df_for_learning.csv')